In [1]:
import torch
import torch_geometric

import identify_x86_graph
import identify_x86_model

In [2]:
print(torch.__version__)

2.0.0


In [3]:
model = identify_x86_model.LightningModel.load_from_checkpoint('lightning_logs/version_51/checkpoints/epoch=4-step=4345.ckpt')

Lightning automatically upgraded your loaded checkpoint from v1.7.7 to v2.0.1.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file lightning_logs/version_51/checkpoints/epoch=4-step=4345.ckpt`


In [4]:
G = identify_x86_graph.load_graph('data/raw/debian/buster/binutils-i686-linux-gnu/usr_bin_i686-linux-gnu-addr2line.graph')

G

Data(num_nodes=25188, x_code=[25188], x_size=[25188], y=[25188], num_edges=278940, edge_index=[2, 278940], edge_type=[278940])

In [5]:
example_input = (G.x_code, G.x_size, G.edge_index, G.edge_type)

with torch.no_grad():
    example_output = model(*example_input)

print(example_output)

tensor([[ 0.7517, -0.7146],
        [ 0.7517, -0.7146],
        [ 0.7517, -0.7146],
        ...,
        [ 0.7517, -0.7146],
        [ 0.7517, -0.7146],
        [ 0.7517, -0.7146]])


In [8]:
model_jit = torch.jit.script(model.model)
print("Model JIT:", model_jit)

Model JIT: RecursiveScriptModule(
  original_name=IdentifyModel
  (model): RecursiveScriptModule(
    original_name=Sequential_4d4c62
    (module_0): RecursiveScriptModule(original_name=Embedding)
    (module_1): RecursiveScriptModule(original_name=Embedding)
    (module_3): RecursiveScriptModule(
      original_name=RGCNConvJittable_4d31db
      (aggr_module): RecursiveScriptModule(original_name=MeanAggregation)
    )
    (module_4): RecursiveScriptModule(original_name=ReLU)
    (module_5): RecursiveScriptModule(
      original_name=RGCNConvJittable_4d3a75
      (aggr_module): RecursiveScriptModule(original_name=MeanAggregation)
    )
    (module_6): RecursiveScriptModule(original_name=ReLU)
    (module_7): RecursiveScriptModule(
      original_name=RGCNConvJittable_4d42a5
      (aggr_module): RecursiveScriptModule(original_name=MeanAggregation)
    )
    (module_8): RecursiveScriptModule(original_name=ReLU)
    (module_9): RecursiveScriptModule(
      original_name=RGCNConvJittable_4

In [9]:
model_jit.save(f"model_jit2.pt")

In [6]:

# torch.onnx.export(
#     model = model,
#     args = (G.x_code, G.x_size, G.edge_index, G.edge_type),
#     f = 'model.onnx',
#     input_names = ['x_code', 'x_size', 'edge_index', 'edge_type'],
#     operator_export_type=torch.onnx.OperatorExportTypes.ONNX,
#     opset_version = 16,
#     verbose = True,
#     dynamic_axes = None,
#     export_modules_as_functions = False,
# )